In [187]:
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re

In [188]:
data = '12/12/2021'
    
def try_format_date(x):
    data_pattern = re.compile("\d{2}/\d{2}/\d{4}")
    if re.match(data_pattern,str(x)):
        data_formated = datetime.strptime(x,"%d/%m/%Y")
        return data_formated
    else:
        return x

try_format_date(data)

datetime.datetime(2021, 12, 12, 0, 0)

In [189]:
df_capital_social = pd.read_excel('historico_composicao_social.xls')
df_capital_social = df_capital_social.drop(columns='%')

for i in range(1,64):
    df_capital_social = df_capital_social.drop(columns=f'%.{i}')

In [190]:
ações_ordinarias = df_capital_social[0:11]
ações_preferenciais = df_capital_social[11:21]
capital_social = df_capital_social[21:31]

In [191]:
ações_ordinarias = ações_ordinarias.transpose()
ações_ordinarias.columns = ações_ordinarias.iloc[0]
ações_ordinarias = ações_ordinarias[1:]
ações_ordinarias['dt_divulgacao'] = ações_ordinarias.index
ações_ordinarias = ações_ordinarias.melt(id_vars=['dt_divulgacao'])
ações_ordinarias = ações_ordinarias.rename(columns={'Capital Social ':'dsc_quadrao_acionario','value':'vl_participacao'})
ações_ordinarias['descricao'] = 'Ações Ordinárias'
ações_ordinarias['dt_divulgacao'] = ações_ordinarias['dt_divulgacao'].apply(lambda x: try_format_date(x))
ações_ordinarias = ações_ordinarias.query("dt_divulgacao != 'Unnamed: 46'", engine='python')
ações_ordinarias['dt_divulgacao'] = ações_ordinarias['dt_divulgacao'].apply(lambda x: datetime.strptime(str(x),"%Y-%m-%d %H:%M:%S").date())
ações_ordinarias['vl_participacao'] = ações_ordinarias['vl_participacao'].apply(lambda x: str(x).rstrip('.0').rstrip(',').replace('.',''))
ações_ordinarias.loc[ações_ordinarias['vl_participacao'] == ''] = 0
ações_ordinarias['vl_participacao'] = ações_ordinarias['vl_participacao'].apply(lambda x: float(x))


In [192]:
ações_preferenciais = ações_preferenciais.transpose()
ações_preferenciais.columns = ações_preferenciais.iloc[0]
ações_preferenciais = ações_preferenciais[1:]
ações_preferenciais['dt_divulgacao'] = ações_preferenciais.index
ações_preferenciais = ações_preferenciais.melt(id_vars=['dt_divulgacao'])
ações_preferenciais = ações_preferenciais.rename(columns={'Capital Social ':'dsc_quadrao_acionario','value':'vl_participacao'})
ações_preferenciais['descricao'] = 'Ações Preferenciais'
ações_preferenciais['dt_divulgacao'] = ações_preferenciais['dt_divulgacao'].apply(lambda x: try_format_date(x))
ações_preferenciais = ações_preferenciais.query("dt_divulgacao != 'Unnamed: 46'", engine='python')
ações_preferenciais['dt_divulgacao'] = ações_preferenciais['dt_divulgacao'].apply(lambda x: datetime.strptime(str(x),"%Y-%m-%d %H:%M:%S").date())
ações_preferenciais['vl_participacao'] = ações_preferenciais['vl_participacao'].apply(lambda x: str(x).rstrip('.0').rstrip(',').replace('.',''))
ações_preferenciais.loc[ações_preferenciais['vl_participacao'] == ''] = 0
ações_preferenciais['vl_participacao'] = ações_preferenciais['vl_participacao'].apply(lambda x: float(x))

In [193]:
capital_social = capital_social.transpose()
capital_social.columns = capital_social.iloc[0]
capital_social = capital_social[1:]
capital_social['dt_divulgacao'] = capital_social.index
capital_social = capital_social.melt(id_vars=['dt_divulgacao'])
capital_social = capital_social.rename(columns={'Capital Social ':'dsc_quadrao_acionario','value':'vl_participacao'})
capital_social['descricao'] = 'Capital Social'
capital_social['dt_divulgacao'] = capital_social['dt_divulgacao'].apply(lambda x: try_format_date(x))
capital_social = capital_social.query("dt_divulgacao != 'Unnamed: 46'", engine='python')
capital_social['dt_divulgacao'] = capital_social['dt_divulgacao'].apply(lambda x: datetime.strptime(str(x),"%Y-%m-%d %H:%M:%S").date())
capital_social['vl_participacao'] = capital_social['vl_participacao'].apply(lambda x: str(x).rstrip('.0').rstrip(',').replace('.',''))
capital_social.loc[capital_social['vl_participacao'] == ''] = 0
capital_social['vl_participacao'] = capital_social['vl_participacao'].apply(lambda x: float(x))

In [195]:
df = pd.concat([ações_ordinarias,ações_preferenciais,capital_social])
df = df.query("dt_divulgacao != 0", engine='python')
df.to_csv('historico_composicao.csv',encoding='utf-8')